## 3.1 Introduction

In Chapter 2, we discussed the network topology, the weights associated with edges, the combination matrix, the average consensus algorithm, and how to implement it in the real CPU clusters with BlueFog. In this section, we will discuss how to solve the general optimization problem in a decentralized manner.

### 3.1.1 The optimization problem

Consider $n$ computing nodes collaborate to solve the problem:

$$\min_{x \in \mathbb{R}^d} \quad \frac{1}{n}\sum_{i=1}^n f_i(x)$$

where $f_i(x)$ is a local and private function held by node $i$. Each node $i$ can access its own variable $x$ or gradient $\nabla f_i(x)$, but it has to communicate to access information from other nodes. 

If each $f_i(x)$ is assumed to be smooth, the leading algorithm to solve the above problem is gradient descent:

\begin{align}
x^{(k+1)} = \frac{1}{n}\sum_{i=1}^n \Big(x^{(k)} - \alpha \nabla f_i(x^{(k)}) \Big)
\end{align}

The above recursion can be conducted in a distributed manner. First, each node $i$ conducts a local gradient descent. Second, all nodes will synchronize with each other to achieve the global average. As a result, the above algorithm is named as **distributed gradient descent**.

The global average operation in distributed gradient descent can be conducted either via Parameter-Server or All-Reduce, but it will incur either a significant bandwidth cost or high latency. This motivates us to solve the above problem in a **decentralized** manner in which each node only needs to communicate with its direct neighbors.

### 3.1.2 Organization

The contents of this chapter is organized as follows:

- Sec.3.2: Decentralized gradient descent: the adapt-then-combine (ATC) version


- Sec.3.3: Decentralized gradient descent: the adapt-with-combination (AWC) version


- Sec.3.4: The influence of network topologies on decentralized gradient descent


- Sec.3.5: Decentralized stochastic gradient descent

### 3.1.3 Initialize BlueFog and test it

All contents in this section are displayed in Jupyter notebook, and all experimental examples are written with BlueFog and iParallel. Readers not familiar with how to run BlueFog in ipython notebook environment is encouraged to read Sec. [HelloWorld section] first. In the following codes, we will initialize BlueFog and test whether it works normally.

In the following code, you should be able to see the id of your CPUs. We use 8 CPUs to conduct the following experiment.

In [1]:
import ipyparallel as ipp

rc = ipp.Client(profile="bluefog")
rc.ids

[0, 1, 2, 3, 4, 5, 6, 7]

Let each agent import necessary modules and then initialize BlueFog. You should be able to see the printed information like:  

> \[stdout:0\] Hello, I am 1 among 8 processes
> 
> ...

In [2]:
%%px
import numpy as np
import bluefog.torch as bf
import torch
from bluefog.common import topology_util
import networkx as nx

bf.init()
print(f"Hello, I am {bf.rank()} among {bf.size()} processes")

[stdout:0] Hello, I am 3 among 8 processes
[stdout:1] Hello, I am 5 among 8 processes
[stdout:2] Hello, I am 6 among 8 processes
[stdout:3] Hello, I am 2 among 8 processes
[stdout:4] Hello, I am 0 among 8 processes
[stdout:5] Hello, I am 1 among 8 processes
[stdout:6] Hello, I am 7 among 8 processes
[stdout:7] Hello, I am 4 among 8 processes


Push seed to each agent so that the simulation can be reproduced.

In [3]:
dview = rc[:]  # A DirectView of all engines
dview.block = True

# Push the data into all workers
dview.push({"seed": 2021}, block=True)

[None, None, None, None, None, None, None, None]

After running the following code, you should be able to see the printed information like 

> \[stdout:0\] I received seed as value:  2021
> 
> ...

In [4]:
%%px
print("I received seed as value: ", seed)

[stdout:0] I received seed as value:  2021
[stdout:1] I received seed as value:  2021
[stdout:2] I received seed as value:  2021
[stdout:3] I received seed as value:  2021
[stdout:4] I received seed as value:  2021
[stdout:5] I received seed as value:  2021
[stdout:6] I received seed as value:  2021
[stdout:7] I received seed as value:  2021


Congratulations! Your BlueFog is initialized and tested successfully.